In [1]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from calibrate_table import calibrate_table
from utils import show
from params import table_aruco_size, K, D

In [ ]:
# visualize table frame
view = "front"
image_file = "data/table_markers/0000.jpg"
image = cv2.imread(image_file)
camera2table, _ = calibrate_table(image, view, K, D, table_aruco_size)
if camera2table is None:
    print("Could not detect table")
else:
    rvec, _ = cv2.Rodrigues(camera2table[0:3, 0:3])
    tvec = camera2table[0:3, 3]
    draw = image.copy()
    cv2.drawFrameAxes(draw, K, D, rvec, tvec, 0.1)
    show(draw)

In [3]:
# get table aruco corners in table frame
view = "front"
folder = "data/table_markers/"
files = sorted(os.listdir(folder))
corners_3d_in_table_all = list()
for f in files:
    image = cv2.imread(folder + f)
    camera2table, corners_3d = calibrate_table(image, view, K, D, table_aruco_size)
    corners_3d = corners_3d[:, 0, 0, :]
    corners_3d = np.expand_dims(np.hstack((corners_3d, np.ones((4, 1)))), axis=-1)
    corners_3d_in_table = np.matmul(np.linalg.inv(camera2table), corners_3d)
    corners_3d_in_table = corners_3d_in_table[:, 0:3, 0]
    corners_3d_in_table_all.append(corners_3d_in_table)

corners_3d_in_table_all = np.array(corners_3d_in_table_all)
corners_3d_in_table_mean = corners_3d_in_table_all.mean(axis=0)


In [4]:
corners_3d_in_table_mean.shape
np.save('data/aruco_corners.npy', corners_3d_in_table_mean)